In [1]:
from sklearn.model_selection import train_test_split

In [2]:
# -*- coding: utf-8 -*-
"""Hackathon.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/19Y6nCWGiR6U0cKHUCMEo1XbPKbVr2xRx

# Importing Libraries
"""

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from wordcloud import WordCloud, STOPWORDS

data = pd.read_csv("Tweets.csv")

data.shape

data.describe()

data = data.loc[:,["text","airline_sentiment"]]

for k,tx in enumerate(data["text"]):
    li = " ".join([word for word in tx.split() if 'http' not in word and not word.startswith('@')and word != 'RT'])
    data.loc[k,"text"] = li

Y = data["airline_sentiment"].apply(lambda x: 0 if x == "negative"  else (1 if (x == "nuetral") else 2))

data["airline_sentiment"].value_counts()

"""# Text mining
1. Removing Stopword, Stemming/lemmatizing, Removing all the symbols
"""

punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''

for k,tx in enumerate(data["text"]):
    res = re.sub(r'[^\w\s]', '', tx)
    data.loc[k,"text"] = ''.join([i.lower() for i in res if not i.isdigit()])

nltk.download("stopwords")

from nltk.corpus import stopwords
stops = stopwords.words('english')

for k,tx in enumerate(data["text"]):
    li = " ".join([word for word in tx.split() if word not in stops])
    data.loc[k,"text"] = li

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
ltz = WordNetLemmatizer()

for k,tx in enumerate(data["text"]):
    li = " ".join([ltz.lemmatize(word) for word in tx.split()])
    data.loc[k,"text"] = li

X = data["text"]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile

In [4]:
def onehot(arr):
    li = np.zeros((3,len(arr)))
    for i,k in enumerate(arr):
        hot = [0,0,0]
        # print(k)
        hot[k] = 1
        li[:,i] = hot
    return li

In [5]:
from  tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint

In [6]:
max_len = max([len(s.split()) for s in data["text"]])

In [7]:
token = Tokenizer()
token.fit_on_texts(data["text"])

In [8]:
vocab_size = len(token.word_index) + 1

# LSTM based Network

In [9]:
X = token.texts_to_sequences(data["text"])
X_pad = pad_sequences(X,maxlen=max_len, padding="post")
Y = data[data.columns[1]]
Y = data["airline_sentiment"].apply(lambda x: 0 if x == "negative"  else (1 if (x == "nuetral") else 2))

In [10]:
X_train,X_test,Y_train,Y_test = train_test_split(X_pad,Y, test_size=0.1, random_state = 10)

In [11]:
Y_train = onehot(Y_train)
Y_test = onehot(Y_test)

Y_test = Y_test.transpose()
Y_train = Y_train.transpose()


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Embedding,LSTM,Bidirectional
from tensorflow.keras.optimizers import Adam

In [13]:
model = Sequential()
model.add(Embedding(vocab_size,100 , input_length=max_len))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.3))
model.add(Dense(3,activation='softmax'))
checkpoint = ModelCheckpoint('model.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='auto')
opt = Adam(learning_rate=0.001)
model.compile(loss = 'categorical_crossentropy', optimizer=opt,metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 21, 100)           1153100   
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                34048     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 1,187,343
Trainable params: 1,187,343
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
history = model.fit(X_train,Y_train, epochs=4, callbacks=[checkpoint],validation_data=(X_test,Y_test), batch_size = 64)

Epoch 1/4
205/206 [============================>.] - ETA: 0s - loss: 0.5124 - accuracy: 0.7607
Epoch 00001: val_accuracy improved from -inf to 0.82309, saving model to model.h5
206/206 [==============================] - 8s 40ms/step - loss: 0.5124 - accuracy: 0.7606 - val_loss: 0.3928 - val_accuracy: 0.8231
Epoch 2/4
205/206 [============================>.] - ETA: 0s - loss: 0.3050 - accuracy: 0.8745
Epoch 00002: val_accuracy improved from 0.82309 to 0.82992, saving model to model.h5
206/206 [==============================] - 7s 36ms/step - loss: 0.3045 - accuracy: 0.8747 - val_loss: 0.3856 - val_accuracy: 0.8299
Epoch 3/4
205/206 [============================>.] - ETA: 0s - loss: 0.2302 - accuracy: 0.9123
Epoch 00003: val_accuracy did not improve from 0.82992
206/206 [==============================] - 7s 35ms/step - loss: 0.2299 - accuracy: 0.9124 - val_loss: 0.4407 - val_accuracy: 0.8238
Epoch 4/4
205/206 [============================>.] - ETA: 0s - loss: 0.1799 - accuracy: 0.9334
Ep

# RNN based Network

In [15]:
from tensorflow.keras.layers import RNN,GRU,SimpleRNN
model = Sequential()
model.add(Embedding(vocab_size,100 , input_length=max_len))
model.add(Bidirectional(SimpleRNN(32)))
# model.add(Dropout(0.3))
model.add(Dense(10))
model.add(Dense(3,activation='softmax'))
checkpoint = ModelCheckpoint('model.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='auto')
opt = Adam(learning_rate=0.001)
model.compile(loss = 'categorical_crossentropy', optimizer=opt,metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 21, 100)           1153100   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                8512      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 33        
Total params: 1,162,295
Trainable params: 1,162,295
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
history = model.fit(X_train,Y_train, epochs=4, callbacks=[checkpoint],validation_data=(X_test,Y_test), batch_size = 64)

Epoch 1/4
206/206 [==============================] - ETA: 0s - loss: 0.4681 - accuracy: 0.7790
Epoch 00001: val_accuracy improved from -inf to 0.81831, saving model to model.h5
206/206 [==============================] - 6s 30ms/step - loss: 0.4681 - accuracy: 0.7790 - val_loss: 0.4038 - val_accuracy: 0.8183
Epoch 2/4
206/206 [==============================] - ETA: 0s - loss: 0.2361 - accuracy: 0.9032
Epoch 00002: val_accuracy did not improve from 0.81831
206/206 [==============================] - 5s 25ms/step - loss: 0.2361 - accuracy: 0.9032 - val_loss: 0.4623 - val_accuracy: 0.8108
Epoch 3/4
205/206 [============================>.] - ETA: 0s - loss: 0.1045 - accuracy: 0.9641
Epoch 00003: val_accuracy did not improve from 0.81831
206/206 [==============================] - 5s 25ms/step - loss: 0.1049 - accuracy: 0.9638 - val_loss: 0.6043 - val_accuracy: 0.7958
Epoch 4/4
204/206 [============================>.] - ETA: 0s - loss: 0.0512 - accuracy: 0.9842
Epoch 00004: val_accuracy did no

# Word2Vec and LSTM based network

In [17]:
sentences = [[] for k in range(len(data["text"]))]
for k,tx in enumerate(data["text"]):
    sentences[k]  = tx.split()
model = Word2Vec(sentences, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
print(model['avoid'])
# save model
model.save('model.bin')
file_name = "word2vec_embeddings.txt"
model.wv.save_word2vec_format(file_name, binary = False)
# load model
new_model = Word2Vec.load('model.bin')

Word2Vec(vocab=11535, size=100, alpha=0.025)
['said', 'plus', 'youve', 'added', 'commercial', 'experience', 'tacky', 'didnt', 'today', 'must', 'mean', 'need', 'take', 'another', 'trip', 'really', 'aggressive', 'blast', 'obnoxious', 'entertainment', 'guest', 'face', 'amp', 'little', 'recourse', 'big', 'bad', 'thing', 'seriously', 'would', 'pay', 'flight', 'seat', 'playing', 'flying', 'va', 'yes', 'nearly', 'every', 'time', 'fly', 'vx', 'ear', 'worm', 'wont', 'go', 'away', 'missed', 'prime', 'opportunity', 'men', 'without', 'hat', 'parody', 'well', 'didntbut', 'amazing', 'arrived', 'hour', 'early', 'youre', 'good', 'know', 'suicide', 'second', 'leading', 'cause', 'death', 'among', 'teen', 'lt', 'pretty', 'graphic', 'much', 'better', 'minimal', 'iconography', 'great', 'deal', 'already', 'thinking', 'nd', 'havent', 'even', 'gone', 'st', 'yet', 'p', 'im', 'fabulous', 'seductive', 'sky', 'u', 'stress', 'travel', 'thanks', 'sfopdx', 'schedule', 'still', 'mia', 'excited', 'first', 'cross', 'co

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [18]:
dic = {}
for tx in data["text"]:
    for k in tx.split():
        if k not in dic:
            dic[k] = len(dic)
embedding_dict = {}
for k in dic.keys():
    embedding_dict[k] = model[k]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [19]:
token = Tokenizer()
token.fit_on_texts(data["text"])
max_len = max([len(s.split()) for s in data["text"]])
vocab_size = len(token.word_index) + 1
X = token.texts_to_sequences(data["text"])
X_pad = pad_sequences(X,maxlen=max_len, padding="post")

In [20]:
word_index = len(token.word_index)
num_words = word_index + 1
embeding_mat = np.zeros((num_words,100))
for word,i in token.word_index.items():
    if(word in embedding_dict):
        embeding_mat[i] = embedding_dict[word]

In [21]:
from tensorflow.keras.initializers import Constant

In [22]:
model = Sequential()
embed = Embedding(num_words,100,embeddings_initializer=Constant(embeding_mat),input_length=max_len,trainable = True)
model.add(embed)
model.add(Bidirectional(LSTM(32)))
# model.add(Dense(10))
model.add(Dense(3,activation='softmax'))
checkpoint = ModelCheckpoint('model_word2vec.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='auto')
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 21, 100)           1153100   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                34048     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 195       
Total params: 1,187,343
Trainable params: 1,187,343
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
history = model.fit(X_train,Y_train, epochs=4, callbacks=[checkpoint],validation_data=(X_test,Y_test), batch_size = 32)

Epoch 1/4
411/412 [============================>.] - ETA: 0s - loss: 0.4813 - accuracy: 0.7782
Epoch 00001: val_accuracy improved from -inf to 0.81284, saving model to model_word2vec.h5
412/412 [==============================] - 15s 36ms/step - loss: 0.4812 - accuracy: 0.7783 - val_loss: 0.4139 - val_accuracy: 0.8128
Epoch 2/4
411/412 [============================>.] - ETA: 0s - loss: 0.3064 - accuracy: 0.8720
Epoch 00002: val_accuracy improved from 0.81284 to 0.82855, saving model to model_word2vec.h5
412/412 [==============================] - 14s 35ms/step - loss: 0.3061 - accuracy: 0.8721 - val_loss: 0.3921 - val_accuracy: 0.8286
Epoch 3/4
411/412 [============================>.] - ETA: 0s - loss: 0.2092 - accuracy: 0.9159
Epoch 00003: val_accuracy did not improve from 0.82855
412/412 [==============================] - 13s 33ms/step - loss: 0.2094 - accuracy: 0.9158 - val_loss: 0.4313 - val_accuracy: 0.8204
Epoch 4/4
411/412 [============================>.] - ETA: 0s - loss: 0.1448 

# Word2Vec + RNN based network    

In [24]:
model = Sequential()
embed = Embedding(num_words,100,embeddings_initializer=Constant(embeding_mat),input_length=max_len,trainable = True)
model.add(embed)
model.add(Bidirectional(SimpleRNN(10)))
# model.add(Dense(10))
model.add(Dense(3,activation='softmax'))
checkpoint = ModelCheckpoint('model_word2vec.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='auto')
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())
history = model.fit(X_train,Y_train, epochs=4, callbacks=[checkpoint],validation_data=(X_test,Y_test), batch_size = 32)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 21, 100)           1153100   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 20)                2220      
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 63        
Total params: 1,155,383
Trainable params: 1,155,383
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/4
412/412 [==============================] - ETA: 0s - loss: 0.5062 - accuracy: 0.7682
Epoch 00001: val_accuracy improved from -inf to 0.78552, saving model to model_word2vec.h5
412/412 [==============================] - 16s 39ms/step - loss: 0.5062 - accuracy: 0.7682 - val_loss: 0.4560 - val_accuracy: 0.7855
Epoch 2/4
411/412 [============================>.]

# Unviersal Encoding Based Classification

In [25]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [26]:
arr_embed = embed(data["text"])

In [27]:
arr_embed.shape

TensorShape([14640, 512])

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(np.array(arr_embed),Y,test_size = 0.05, random_state= 10)

In [29]:
X_train.shape

(13908, 512)

In [30]:
Y_train = onehot(Y_train)
Y_test = onehot(Y_test)

Y_test = Y_test.transpose()
Y_train = Y_train.transpose()

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
model = Sequential()
model.add(Dense(300, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.9))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(10,activation="relu"))
model.add(Dense(3,activation= "sigmoid"))
# opt =Adam(learning_rate=0.01)
checkpoint = ModelCheckpoint('use_model.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='auto')
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

history = model.fit(X_train, Y_train,validation_data = (X_test,Y_test), epochs=20,callbacks=[checkpoint], batch_size=32)


Epoch 1/20
423/435 [============================>.] - ETA: 0s - loss: 0.6133 - accuracy: 0.7081
Epoch 00001: val_accuracy improved from -inf to 0.81831, saving model to use_model.h5
435/435 [==============================] - 2s 4ms/step - loss: 0.6106 - accuracy: 0.7100 - val_loss: 0.4136 - val_accuracy: 0.8183
Epoch 2/20
434/435 [============================>.] - ETA: 0s - loss: 0.4476 - accuracy: 0.8049
Epoch 00002: val_accuracy improved from 0.81831 to 0.82923, saving model to use_model.h5
435/435 [==============================] - 2s 4ms/step - loss: 0.4476 - accuracy: 0.8049 - val_loss: 0.3939 - val_accuracy: 0.8292
Epoch 3/20
424/435 [============================>.] - ETA: 0s - loss: 0.4279 - accuracy: 0.8167
Epoch 00003: val_accuracy improved from 0.82923 to 0.83470, saving model to use_model.h5
435/435 [==============================] - 2s 4ms/step - loss: 0.4274 - accuracy: 0.8169 - val_loss: 0.3925 - val_accuracy: 0.8347
Epoch 4/20
425/435 [============================>.] - E

In [32]:
model.load_weights("use_model.h5")
model.evaluate(X_test,Y_test, verbose=True)

23/23 [==============================] - 0s 2ms/step - loss: 0.3698 - accuracy: 0.8429


[0.36983728408813477, 0.8428961634635925]

#Highest Accuracies 
1. 85.65% USE based model
2. 84% word2vec LSTM 
3. 83.8 Only LSTM